In [ ]:
import os
import leafmap
import numpy as np

In [ ]:
os.environ["NODATA"] = "-32768"

In [ ]:
m =  leafmap.Map()
m.add_basemap("OpenTopoMap")
m.add_basemap("USGS 3DEP Elevation")
m.add_basemap("USGS Hydrography"
              )
m

In [ ]:
lat = 44.36169
lon = -122.82802

m = leafmap.Map(center = [lat, lon], zoom = 10)
m.add_marker([lat, lon])
m

In [ ]:
geometry = {"x": lon , "y": lat}

In [ ]:
gdf = leafmap.get_wbd(geometry, digit=10, return_geometry=True)
gdf.explore()

In [ ]:
gdf.to_file("basin.geojson")

In [ ]:
array = leafmap.get_3dep_dem(
    gdf,
    resolution=30,
    output = "dem.tif",
    dst_crs = "EPSG:3857",
    to_cog=True,
    overwrite=True
,)
array

In [ ]:
m.add_raster("dem.tif", palette="terrain", nodata=np.nan, layer_name="DEM")
m

In [ ]:
metadata =leafmap.image_metadata("dem.tif")
metadata

In [ ]:
leafmap.image_min_max("dem.tif")

In [ ]:
m.add_colormap(cmap="terrain", vmin="60", vmax=1500, label="Elevation (m)")

In [ ]:
wbt = leafmap.WhiteboxTools()

In [ ]:
wbt.version()

In [ ]:
leafmap.whiteboxgui()

In [ ]:
wbt.set_working_dir(os.getcwd())
wbt.verbose = False

In [ ]:
wbt.feature_preserving_smoothing("dem.tif", "smoothed.tif", filter=9  )

In [ ]:
m = leafmap.Map()
m.add_basemap("Satellite")
m.add_raster("smoothed.tif", colormap="terrain", layer_name="Smoothed DEM")
m.add_geojson("basin.geojson", layer_name="Watershed", info_mode=None)
m.add_basemap("USGS Hydrography", show=False)
m

In [ ]:
wbt.hillshade("smoothed.tif", "hilllshade.tif", azimuth=315, altitude=35)

In [ ]:
m.add_raster("hilllshade.tif", layer_name="Hillshade")
m.layers[-1].opacity = 0.6

In [ ]:
wbt.find_no_flow_cells("smoothed.tif", "noflow.tif")

In [ ]:

m.add_raster("noflow.tif", layer_name="No Flow Cells")

In [ ]:
wbt.fill_depressions("smoothed.tif", "filled.tif")

In [ ]:
wbt.breach_depressions("smoothed.tif", "breached.tif")

In [ ]:
wbt.find_no_flow_cells("breached.tif", "noflow2.tif")

In [ ]:
m.layers[-1].visible = False
m.add_raster("noflow2.tif", layer_name="No Flow Cells after Breaching")
m

In [ ]:
wbt.d8_pointer("breached.tif", "flow_direction.tif")

In [ ]:
wbt.d8_flow_accumulation("breached.tif", "flow_accum.tif")

In [ ]:
m.add_raster("flow_accum.tif", layer_name="Flow Accumulation")

In [ ]:
wbt.extract_streams("flow_accum.tif", "streams.tif", threshold=5000)

In [ ]:
m.layers[-1].visible = False
m.add_raster("streams.tif", layer_name="Streams")

In [ ]:
wbt.distance_to_outlet(
    "flow_direction.tif", streams="streams.tif", output="distance_to_outlet.tif"
)

In [ ]:
m.add_raster("distance_to_outlet.tif", layer_name="Distance to Outlet")

In [ ]:
wbt.raster_streams_to_vector(
    "streams.tif", d8_pntr="flow_direction.tif", output="streams.shp"
)

In [ ]:
leafmap.vector_set_crs(source="streams.shp", output="streams.shp", crs="EPSG:3857")

In [ ]:
m.add_shp(
    "streams.shp",
    layer_name="Streams Vector",
    style={"color": "#ff0000", "weight": 3},
    info_mode=None,
)
m

In [ ]:
if m.user_roi is not None:
    m.save_draw_features("pour_point.shp", crs="EPSG:3857")
else:
    lat = 44.284642
    lon = -122.611217
    leafmap.coords_to_vector([lon, lat], output="pour_point.shp", crs="EPSG:3857")
    m.add_marker([lat, lon])

In [ ]:
wbt.snap_pour_points(
    "pour_point.shp", "flow_accum.tif", "pour_point_snapped.shp", snap_dist=300
)

In [ ]:
m.add_shp("pour_point_snapped.shp", layer_name="Pour Point", info_mode=False)

In [ ]:
wbt.watershed("flow_direction.tif", "pour_point_snapped.shp", "watershed.tif")

In [ ]:

m.add_raster("watershed.tif", layer_name="Watershed")
m

In [ ]:
wbt.raster_to_vector_polygons("watershed.tif", "watershed.shp")

In [ ]:
m.layers[-1].visible = False
m.add_shp(
    "watershed.shp",
    layer_name="Watershed Vector",
    style={"color": "#ffff00", "weight": 3},
    info_mode=False,
)

In [ ]:

import leafmap

In [ ]:

wbt = leafmap.WhiteboxTools()
wbt.set_working_dir(os.getcwd())
wbt.verbose = False

In [ ]:
url = "https://github.com/opengeos/datasets/releases/download/lidar/madison.zip"
filename = "madison.las"

In [ ]:
leafmap.download_file(url, "madison.zip", quiet=True)

In [ ]:

laz = leafmap.read_lidar(filename)

In [ ]:


laz

In [ ]:
str(laz.header.version)

In [ ]:
las = leafmap.convert_lidar(laz, file_version="1.4")

In [ ]:
str(las.header.version)

In [ ]:
leafmap.write_lidar(las, "madison.las")

In [ ]:
wbt.lidar_histogram("madison.las", "histogram.html")

In [ ]:
leafmap.view_lidar("madison.las")

In [ ]:
wbt.lidar_elevation_slice("madison.las", "madison_rm.las", minz=0, maxz=450)

In [ ]:
leafmap.view_lidar("madison_rm.las", cmap="terrain")

In [ ]:
wbt.lidar_digital_surface_model(
    "madison_rm.las", "dsm.tif", resolution=1.0, minz=0, maxz=450
)

In [ ]:
leafmap.add_crs("dsm.tif", epsg=2255)

In [ ]:
m = leafmap.Map()
m.add_basemap("Satellite")
m.add_raster("dsm.tif", colormap="terrain", layer_name="DSM")
m

In [ ]:
wbt.remove_off_terrain_objects("dsm.tif", "dem.tif", filter=25, slope=15.0)

In [ ]:
m.add_raster("dem.tif", palette="terrain", layer_name="DEM")
m

In [ ]:
chm = wbt.subtract("dsm.tif", "dem.tif", "chm.tif")

In [ ]:
m.add_raster("chm.tif", palette="gist_earth", layer_name="CHM")
m.add_layer_manager()
m